In [206]:
from PIL import Image
import numpy as np

In [207]:
def getblocks(image: np.ndarray, blockshape: tuple, moveAxis: bool = True, info: bool = False, addChannel: bool = True) -> np.ndarray:
    '''
    takes the array of image in grey= 2D and in RGB = 3D
    takes the numpy array and converts it the the blocks in the fastest way
    '''
    if(info):
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)

    oldshape = list(image.shape)
    if addChannel and len(image.shape) == 2:
        mode = "grey"
        image = image.reshape((*image.shape, 1))
    else:
        mode = "color"

    if addChannel:
        img_height, img_width, channels = image.shape
    else:
        img_height, img_width = image.shape

    tile_height, tile_width = blockshape

    if addChannel:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels
    else:
        shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width

    def printinfo():
        print("Old Shape:", oldshape)
        print("Image Shape:", image.shape)
        print("Block Shape:", blockshape)
        print("New Shape Initial:", shp)
        print("img_height % tile_height != 0 :", img_height % tile_height != 0)
        print("img_width % tile_width != 0 :", img_width % tile_width != 0)

    if img_height % tile_height != 0 or img_width % tile_width != 0:
        print("warning: Block size is not fit for the image!")
        printinfo()

    if(info):
        printinfo()

    tiled_array = image.reshape(shp)
    tiled_array = tiled_array.swapaxes(1, 2)

    if moveAxis:
        if(addChannel):
            tiled_array = tiled_array.reshape(-1,
                                              *(tile_height, tile_width, channels))
            tiled_array = np.moveaxis(tiled_array, source=len(
                tiled_array.shape)-1, destination=1)
        else:
            tiled_array = tiled_array.reshape(-1, *(tile_height, tile_width))

    return tiled_array


In [208]:
def combineBlocks(tiled_array: np.ndarray, imageshape: tuple, blockshape: tuple, movedAxis: bool = True, channel: bool = True) -> np.ndarray:

    if channel:
        if len(imageshape) == 2:
            mode = "grey"
            imageshape = *imageshape, 1
        else:
            mode = "color"

    if channel:
        img_height, img_width, channels = imageshape
    else:
        img_height, img_width = imageshape

    tile_height, tile_width = blockshape

    if movedAxis:
        image = tiled_array.copy()
        if(channel):
            image=np.moveaxis(image, source=1, destination=-1)
            image = image.reshape(img_height//tile_height, tile_height,
                                  img_width//tile_width, tile_width, channels)
            swapaxisShape = list(image.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            image = image.reshape(swapaxisShape)
            image = image.swapaxes(1, 2)
        else:
            f = image.reshape(img_height//tile_height, tile_height,
                              img_width//tile_width, tile_width)
            swapaxisShape = list(f.shape)
            swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
            tmp = f.reshape(swapaxisShape)
            image = tmp.swapaxes(1, 2)
    else:
        image = tiled_array
        # I haven't completed this else case. Btw we aren't using this case lol :)

    return image.reshape(imageshape)


In [209]:
image = np.arange(1,(512*1024*3)+1).reshape(512,1024,3)
image.shape,image.strides,image

((512, 1024, 3),
 (24576, 24, 8),
 array([[[      1,       2,       3],
         [      4,       5,       6],
         [      7,       8,       9],
         ...,
         [   3064,    3065,    3066],
         [   3067,    3068,    3069],
         [   3070,    3071,    3072]],
 
        [[   3073,    3074,    3075],
         [   3076,    3077,    3078],
         [   3079,    3080,    3081],
         ...,
         [   6136,    6137,    6138],
         [   6139,    6140,    6141],
         [   6142,    6143,    6144]],
 
        [[   6145,    6146,    6147],
         [   6148,    6149,    6150],
         [   6151,    6152,    6153],
         ...,
         [   9208,    9209,    9210],
         [   9211,    9212,    9213],
         [   9214,    9215,    9216]],
 
        ...,
 
        [[1563649, 1563650, 1563651],
         [1563652, 1563653, 1563654],
         [1563655, 1563656, 1563657],
         ...,
         [1566712, 1566713, 1566714],
         [1566715, 1566716, 1566717],
         [15

In [210]:
blocksize=(16,16)

In [211]:
a=getblocks(image, blockshape=blocksize)
a.shape, a.strides, a

((2048, 3, 16, 16),
 (6144, 8, 384, 24),
 array([[[[      1,       4,       7, ...,      40,      43,      46],
          [   3073,    3076,    3079, ...,    3112,    3115,    3118],
          [   6145,    6148,    6151, ...,    6184,    6187,    6190],
          ...,
          [  39937,   39940,   39943, ...,   39976,   39979,   39982],
          [  43009,   43012,   43015, ...,   43048,   43051,   43054],
          [  46081,   46084,   46087, ...,   46120,   46123,   46126]],
 
         [[      2,       5,       8, ...,      41,      44,      47],
          [   3074,    3077,    3080, ...,    3113,    3116,    3119],
          [   6146,    6149,    6152, ...,    6185,    6188,    6191],
          ...,
          [  39938,   39941,   39944, ...,   39977,   39980,   39983],
          [  43010,   43013,   43016, ...,   43049,   43052,   43055],
          [  46082,   46085,   46088, ...,   46121,   46124,   46127]],
 
         [[      3,       6,       9, ...,      42,      45,      48],


In [212]:
res=combineBlocks(a, image.shape, blocksize)

In [213]:
(res.flat==image.flat).all()

True

In [ ]:
img_height, img_width, channels = image.shape

In [ ]:
tile_height, tile_width = blocksize

dividing

In [ ]:
img_height, img_width, channels = image.shape

In [ ]:
shp = img_height//tile_height, tile_height, img_width//tile_width, tile_width, channels
shp

In [ ]:
if img_height % tile_height != 0 or img_width % tile_width != 0:
  print("warning: Block size is not fit for the image!")

In [ ]:
u=image.reshape(shp)
u.shape, u.strides, u

In [ ]:
v=u.swapaxes(1,2)
v.shape, v.strides, v

In [ ]:
x=v.reshape(-1, *(tile_height, tile_width, channels))
x.shape, x.strides, x

In [ ]:
y=np.moveaxis(x, source=len(x.shape)-1, destination=1)
y.shape, y.strides, y

combining

In [ ]:
b=y.copy()
b.shape, b.strides, b

In [ ]:
b1=np.moveaxis(b, source=1, destination=-1)
b1.shape, b1.strides, b1

In [ ]:
c = b1.reshape(img_height//tile_height, tile_height,
                                  img_width//tile_width, tile_width, channels)
c.shape, c.strides, c

In [ ]:
swapaxisShape = list(c.shape)
swapaxisShape

In [ ]:
swapaxisShape[1], swapaxisShape[2] = swapaxisShape[2], swapaxisShape[1]
swapaxisShape

In [ ]:
d = c.reshape(swapaxisShape)
d.shape, d.strides, d

In [ ]:
e=d.swapaxes(1,2)
e.shape, e.strides, e

In [ ]:
f=e.reshape( image.shape)

In [ ]:
(f.flat==image.flat).all()